In [ ]:
import tkinter as tk
from tkinter import messagebox
from queue import Queue, PriorityQueue

# --- Hàm kiểm tra an toàn có thể chèn hậu ---
def isSafe(vitri, row, col):
    for r, c in enumerate(vitri):
        if c == col or abs(c - col) == abs(r - row):
            return False
    return True


# --- Thuật toán tìm kiếm  ---
def bfs_8queens_steps():
    n = 8
    q = Queue()
    q.put([])
    while not q.empty():
        vitri = q.get()
        row = len(vitri)
        if row == n:
            yield vitri
            return
        for col in range(n):
            if isSafe(vitri, row, col):
                q.put(vitri + [col])
                yield vitri + [col]


def dfs_8queens_steps():
    n = 8
    stack = [[]]
    while stack:
        vitri = stack.pop()
        row = len(vitri)
        if row == n:
            yield vitri
            return
        for col in range(n-1, -1, -1):
            if isSafe(vitri, row, col):
                stack.append(vitri + [col])
                yield vitri + [col]


def ucs_8queens_steps():
    n = 8
    pq = PriorityQueue()
    pq.put((0, []))
    while not pq.empty():
        cost, vitri = pq.get()
        row = len(vitri)
        if row == n:
            yield vitri
            return
        for col in range(n):
            if isSafe(vitri, row, col):
                pq.put((cost + 1, vitri + [col]))
                yield vitri + [col]


# --- GUI ---
root = tk.Tk()
root.title("8 Queens - BFS/DFS/UCS")
root.geometry("1200x700+100+30")
root.configure(bg="#BDE7E7")

title_label = tk.Label(root, text="8 QUEENS SEARCH (BFS / DFS / UCS)",
                       font=("SegoeUI", 20, "bold"),
                       fg="#0E2846", bg="#BDE7E7")
title_label.pack(pady=10)

main_frame = tk.Frame(root, bg="#BDE7E7")
main_frame.pack(fill="both", expand=True, padx=10, pady=10)

# Bàn cờ trống (bên trái)
board = tk.Frame(main_frame, width=480, height=480)
board.pack(side="left", padx=20)
board.pack_propagate(False)

for row in range(8):
    for col in range(8):
        color = "#47C0C0" if (row + col) % 2 == 0 else "#17375C"
        cell = tk.Frame(board, width=60, height=60, bg=color)
        cell.grid(row=row, column=col)

# Bàn cờ hiển thị thuật toán (bên phải)
placedFrame = tk.Frame(main_frame, width=480, height=480)
placedFrame.pack(side="right", padx=20)
placedFrame.pack_propagate(False)


def draw_empty_board():
    for widget in placedFrame.winfo_children():
        widget.destroy()
    cells = []
    for row in range(8):
        row_cells = []
        for col in range(8):
            color = "#47C0C0" if (row + col) % 2 == 0 else "#17375C"
            cell = tk.Frame(placedFrame, width=60, height=60, bg=color)
            cell.grid(row=row, column=col)
            cell.pack_propagate(False)
            row_cells.append(cell)
        cells.append(row_cells)
    return cells


def draw_solution(cells, solution):
    for row in range(8):
        for col in range(8):
            for widget in cells[row][col].winfo_children():
                widget.destroy()
    for row, col in enumerate(solution):
        lbl = tk.Label(cells[row][col], text="\u2655", fg="red",
                       bg=cells[row][col]["bg"], font=("Arial", 24, "bold"))
        lbl.pack(expand=True, fill="both")


# --- Biến toàn cục ---
cells_global = draw_empty_board()
search_generator = None
delay = 100  # ms
algo_choice = tk.StringVar(value="BFS")
skip_delay = False
running = False  


def run_step():
    global search_generator, cells_global, skip_delay, running
    if not running:
        return
    try:
        if skip_delay:  # nếu skip thì chạy hết đến nghiệm
            for state in search_generator:
                pass
            cells_global = draw_empty_board()
            draw_solution(cells_global, state)
            skip_delay = False
            running = False
            return
        else:
            state = next(search_generator)
            cells_global = draw_empty_board()
            draw_solution(cells_global, state)
            root.after(delay, run_step)
    except StopIteration:
        messagebox.showinfo("Thông báo", "Đã tìm được nghiệm!")
        running = False


def start_search():
    global search_generator, cells_global, running
    cells_global = draw_empty_board()
    choice = algo_choice.get()
    if choice == "BFS":
        search_generator = bfs_8queens_steps()
    elif choice == "DFS":
        search_generator = dfs_8queens_steps()
    else:
        search_generator = ucs_8queens_steps()
    running = True
    run_step()


def skip_search():
    global skip_delay, running
    if not running:
        return
    skip_delay = True
    run_step()


def reset_board():
    global search_generator, running, skip_delay
    running = False
    skip_delay = False
    search_generator = None
    draw_empty_board()


def quit_app():
    root.destroy()


# --- Control panel ---
control_label = tk.Frame(root, bg="#BDE7E7")
control_label.pack(pady=5)

tk.Radiobutton(control_label, text="BFS", variable=algo_choice,
               value="BFS", bg="#BDE7E7").pack(side="left", padx=5)
tk.Radiobutton(control_label, text="DFS", variable=algo_choice,
               value="DFS", bg="#BDE7E7").pack(side="left", padx=5)
tk.Radiobutton(control_label, text="UCS", variable=algo_choice,
               value="UCS", bg="#BDE7E7").pack(side="left", padx=5)

control_panel = tk.Frame(root, bg="#BDE7E7")
control_panel.pack(pady=10)
btn_start = tk.Button(control_panel, text="Start",
                      font=("SegoeUI", 14, "bold"),
                      width=10, command=start_search, bg ="#BDE7E7")
btn_start.pack(side="left", padx=20)

btn_skip = tk.Button(control_panel, text="Skip",
                     font=("SegoeUI", 14, "bold"), bg ="#BDE7E7",
                     width=10, command=skip_search)
btn_skip.pack(side="left", padx=10)

btn_reset = tk.Button(control_panel, text="Reset", bg ="#BDE7E7",
                      font=("SegoeUI", 14, "bold"),
                      width=10, command=reset_board)
btn_reset.pack(side="left", padx=10)



root.mainloop()
